In [5]:
# #Example to show that it now also works with just a single column or single row
# mat <- matrix(1:100, byrow=T, nrow=10)
# column_annotation <- sample(c("red", "blue", "green"), 10, replace=T)
# column_annotation <- as.matrix(column_annotation)
# colnames(column_annotation) <- c("Variable X")

# row_annotation <- sample(c("red", "blue", "green"), 10, replace=T)
# row_annotation <- as.matrix(t(row_annotation))
# rownames(row_annotation) <- c("Variable Y")

# heatmap.3(mat, RowSideColors=row_annotation, ColSideColors=column_annotation)

# XGBoost classifier python code
def expand_grid(dictionary):
    return pd.DataFrame([row for row in product(*dictionary.values())],columns=dictionary.keys())





    #max_delta_step 1-10 might help logistic regression
    #scale_pos_weight [default=1] control the balance of positive and negative weights, useful for unbalanced classes. A typical value to consider: sum(negative instances) / sum(positive instances
    #sum(Y5_trn==0)/sum(Y5_trn==1)     Out[1725]: 0.46522585270920824
    # Prediction: prob of class 1  (cutoff of 0.5, 0.1 -> class0, 0.9 -> class1)
    obj='binary:logistic'
    #nthread=4
    num_cv=5

    grid = {'eta': [0.001, 0.01], #seq(2, 10, 1)/ntrees,
        'max_depth': [4, 6, 8],
        'reg_lambda': [1, 5, 10],
        #'min_child_weight': [1, 5],
        'colsample_bytree': [0.5, 0.75, 1],
        'subsample': [0.4, 0.6, 0.8, 1],
        'max_delta_step': [1, 5, 10],
        'scale_pos_weight': [0.5, 1, 1.5]}
    
    '''
    grid = {'eta': [0.01], #seq(2, 10, 1)/ntrees,
        'max_depth': [4],
        'reg_lambda': [1],
        'colsample_bytree': [0.5],
        'subsample': [0.4],
        'max_delta_step': [1],
        'scale_pos_weight': [0.5]}
    '''    
    
    param_grid=expand_grid(grid)  # Expand grid
    num_param=param_grid.shape[0] # Get number of parameters
    
    out_name=out_loc+in_name+'_xgb_param_grid.csv'
    param_grid.to_csv(out_name, index=False)
    
                                    
    # Get performance results: validation & CV training of the training partitions
    res5_df = pd.DataFrame(columns=['val_aucpr','val_acc','val_tn','val_fp','val_fn','val_tp',
                                 'cv_aucpr','cv_acc','cv_tn','cv_fp','cv_fn','cv_tp'])
    max5_auc = 0
    best5_param = pd.DataFrame()
    best5_res = pd.DataFrame()
    
    
    # Parameter tuning using CV
    for i, row in param_grid.iterrows():
        print('--- Processing parameter ',i+1,'of',num_param,'(',round((i+1)/num_param*100,2),'%) ---')
        
        xgb = XGBClassifier(objective=obj, n_estimators=5000,                       
                            eta=row['eta'], 
                            max_depth = int(row['max_depth']), 
                            reg_lambda = row['reg_lambda'], 
                            colsample_bytree = row['colsample_bytree'], 
                            subsample = row['subsample'], 
                            max_delta_step = row['max_delta_step'],
                            scale_pos_weight = row['scale_pos_weight'])
        
    
        # Cross-validation (Stratified)
        kf = StratifiedKFold(n_splits=num_cv)
        #-- CV: AHI5 --
        for cv_idx, val_idx in kf.split(X5_trn, Y5_trn):
            X5_cv, Y5_cv   = X5_trn.iloc[cv_idx,:], Y5_trn.iloc[cv_idx]
            X5_val, Y5_val = X5_trn.iloc[val_idx,:], Y5_trn.iloc[val_idx]
            eval_set = [(X5_val, Y5_val)]
            
            #xgb.fit(X5_cv, Y5_cv, eval_set=eval_set, early_stopping_rounds=5, eval_metric="logloss", verbose=False)
            #xgb.fit(X5_cv, Y5_cv, eval_set=eval_set, early_stopping_rounds=5, eval_metric="map", verbose=False)
            xgb.fit(X5_cv, Y5_cv, eval_set=eval_set, early_stopping_rounds=5, eval_metric="aucpr", verbose=False)
            
            # Probability (AUC for precision & recall)
            Yhat5_val = xgb.predict_proba(X5_val)[:,1] # prob of class 1
            Yhat5_cv = xgb.predict_proba(X5_cv)[:,1]   # prob of class 1
        
            pre_val, rec_val, thre = precision_recall_curve(Y5_val, Yhat5_val) # a vector
            pre_cv, rec_cv, thre = precision_recall_curve(Y5_cv, Yhat5_cv)     # a vector
        
            auc_val = auc(rec_val, pre_val) # a value
            auc_cv = auc(rec_cv, pre_cv)    # a value
        
            # Binary (TN, FP, FN, TP & Acc)
            Yhat5_val = xgb.predict(X5_val) # binary prediction {0,1}
            Yhat5_cv = xgb.predict(X5_cv)  # binary prediction {0,1}
            
            acc_val = accuracy_score(Y5_val, Yhat5_val)
            acc_cv = accuracy_score(Y5_cv, Yhat5_cv)
        
            tn_val, fp_val, fn_val, tp_val = confusion_matrix(Y5_val, Yhat5_val).ravel()
            tn_cv, fp_cv, fn_cv, tp_cv = confusion_matrix(Y5_cv, Yhat5_cv).ravel()
            
            res5_df.loc[i] = np.around([auc_val, acc_val, tn_val, fp_val, fn_val, tp_val,
                                        auc_cv, acc_cv, tn_cv, fp_cv, fn_cv, tp_cv], 3)
    
#       Get current best model. 
#       Note: should consider using average of 5fold CV's best
            if auc_val > max5_auc:
                best5_param = pd.DataFrame(row)
                best5_res = res5_df.loc[i]
                max5_auc = auc_val
    
    
    

out_name=out_loc+in_name+'_X5_trn_xgb_param_performance.csv'
res5_df.to_csv(out_name, header=True, index=True)
out_name=out_loc+in_name+'_X5_trn_xgb_param_bestset.csv'
best5_param.to_csv(out_name, header=True, index=True)
out_name=out_loc+in_name+'_X5_trn_xgb_param_best_res.csv'
best5_res.to_csv(out_name, header=True, index=True)

NameError: name 'out_loc' is not defined